<a href="https://colab.research.google.com/github/ChrisM1101/MSc/blob/main/pi_mc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nikolaos Stergioulas, Aristotle University of Thessaloniki
#
# Content provided under a Creative Commons Attribution license, CC BY-NC-SA 4.0; code under GNU GPLv3 License.
# (c)2024 Nikolaos Stergioulas

import subprocess
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Compile the C code
subprocess.run(["gcc", "-O3", "-fopenmp", "pi_mc.c", "-o", "pi_mc"])

# List of number of threads
num_threads_list = [1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32]

# List to store execution times
execution_times = []